# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0)
# I chose 0 because I want the model to be deterministic and not generate random responses


In [8]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
    "I want to buy a {product} but I am not sure which one to buy. Can you help me with that?"
)

In [9]:

chain = LLMChain(llm=llm, prompt=prompt)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
product = "eGPU Nvidia"
chain.run(product)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"Of course! There are several factors to consider when choosing an eGPU, such as compatibility with your computer, the graphics card model, and your budget. \n\nSome popular eGPU enclosures that are compatible with Nvidia graphics cards include the Razer Core X, Akitio Node, and Sonnet eGFX Breakaway Box. \n\nAs for the graphics card itself, some popular Nvidia models for eGPUs include the Nvidia GeForce RTX 2080 Ti, RTX 2070 Super, and GTX 1660 Ti. \n\nIt's also important to consider your budget and what you will be using the eGPU for. If you are a gamer looking for high performance, you may want to invest in a higher-end graphics card. However, if you are using the eGPU for more basic tasks such as video editing or graphic design, a mid-range card may be sufficient. \n\nUltimately, it's best to do some research and read reviews to determine which eGPU and graphics card combination will best suit your needs."

## SimpleSequentialChain

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
     "I want to buy a {product} but I am not sure which one to buy. Can you help me with that?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [13]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    "Considering that {considerations}, I choose for you: [recomendation]"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [14]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [15]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Of course! Here are a few popular eGPU options from Nvidia that you can consider:

1. Nvidia GeForce RTX 3080: This high-end eGPU offers excellent performance and is ideal for gaming and video editing.

2. Nvidia GeForce RTX 2070 Super: This mid-range eGPU is a good option for those looking for a balance of performance and affordability.

3. Nvidia Quadro RTX 5000: This professional-grade eGPU is designed for creative professionals who need powerful graphics capabilities for tasks such as 3D rendering and video editing.

Ultimately, the best eGPU for you will depend on your specific needs and budget. I recommend doing some research on each of these options and reading reviews to determine which one would be the best fit for you.
Nvidia GeForce RTX 3080.

> Finished chain.


'Nvidia GeForce RTX 3080.'

**Repeat the above twice for different products**

## SequentialChain

In [16]:
from langchain.chains import SequentialChain

In [17]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  "Translate the following review: {review} to Chinese"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="chinese_review" #Give a name to your output
                    )


In [18]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    "Summarize the following review: {review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary_review" #give a name to this output
                    )


In [19]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review: {summary_review} to English"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="english_summary"
                      )


In [20]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "The summary of the review is: {summary_review}. The translation of the summary is: {english_summary}. Make a follow up message."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up_message"
                     )


In [21]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["review", "summary_review", "follow_up_message", "chinese_review"],
    verbose=True
)

In [22]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...


/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'summary_review': 'The reviewer finds the taste mediocre and the foam does not last, which they find strange. They suggest that the product may be old or a counterfeit, as they have purchased the same item from stores and found the taste to be much better.',
 'follow_up_message': 'Hello! Thank you for your feedback on the product. We are sorry to hear that you have had a negative experience with it. We take the quality of our products very seriously and would like to address your concerns. If you could provide us with more information, such as the batch number or where you purchased the item, we can investigate further. Our goal is to ensure that all of our customers are satisfied with their purchases. Thank you for bringing this to our attention.',
 'chinese_review': '我觉得味道很差。泡沫不持久，很奇怪。我在商店里买到的相同产品味道好多了...是老产

**Repeat the above twice for different products or reviews**

## Router Chain

In [23]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [35]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "default",
        "description": "Good for answering questions about other topics",
        "prompt_template": "{input}"
    }
]

In [36]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [37]:
llm = ChatOpenAI(temperature=0)

In [38]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [39]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [40]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [41]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [42]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )

In [43]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [44]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [45]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**